In [1]:
### 已经学习 5(数据处理) 6(数据筛选) 7(数值操作)章
### 7章 数值替换(replace)  数据排序(sort_values)  数据删除 df.drop(["c2","c3"],axis = 1) 和条件删除 df[df["c2"] > 10]
### df[df["c3"].isin([20,21])] 保留c3列包含20和21的数据
### 数值计数 value_counts  计算某一个列,每一个值出现的次数
### 某列的唯一值 unique
### 数值查找,相当于contain,显示该数值是否在该表存在,或者该列存在,isin
### 插入新列
### 数据转置  df.T
### apply 与 applymap函数
### cut qcut 数据分组拆分

### 7.3节 rank的method = min,max,first,average 7.11 df.stack df.unstack 7.12 melt 暂时忽略不讲

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
from pandas.tseries.offsets import Day,Hour,Minute
import sys
stdo = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout= stdo

In [3]:
def load_data(filepath):
    df = pd.read_csv(filepath,sep=',',nrows = 100,encoding = "utf-8",engine="python")
    return df

In [4]:
df = load_data('/Users/maming/Downloads/test/dataframe_2_data_deal.txt')
df['c4'] = pd.to_datetime(df['c4'], format='%Y%m%d %H:%M:%S')  ## 字符串转换成datetime64
df['c6'] = df['c6'].apply(lambda x: datetime.strptime(x,'%Y%m%d %H:%M:%S'))  ## 字符串转换成datetime64
df['c7'] = df['c7'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d'))  ## 字符串转换成datetime64
df

,c1,c2,c3,c4,c5,c6,c7
0,0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
1,1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
2,3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03
3,NaN,NaN,NaN,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [5]:
### 缺失值的确认 以及 填充,
### NaN NULL null 都表示缺失值
df.info() ### 判断列是否有NaN数据
print df.isnull() ### 返回true/false 表示是否是null.数据多的时候不太常用,可以用df.info()查看是否有null数据
### 缺失值删除
print df.dropna() ### 删除有缺失值的整行数据---NaN
print df.dropna(how = "all") ### 整行都是缺失值--空白行才会被删除
### 缺失值填充
### print df.fillna(0) ### 缺失值都填充为0
print df.fillna({"c1":-99,"c2":-999}) ## 针对不同的列 设置不同的缺失值


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
c1    3 non-null float64
c2    3 non-null float64
c3    3 non-null float64
c4    4 non-null datetime64[ns]
c5    4 non-null object
c6    4 non-null datetime64[ns]
c7    4 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(3), object(1)
memory usage: 296.0+ bytes
      c1     c2     c3     c4     c5     c6     c7
0  False  False  False  False  False  False  False
1  False  False  False  False  False  False  False
2  False  False  False  False  False  False  False
3   True   True   True  False  False  False  False
    c1    c2    c3                  c4          c5                  c6  \
0  0.0  10.0  20.0 2019-09-04 22:34:21  2019-09-03 2019-09-03 22:34:00   
1  1.0  11.0  21.0 2019-09-04 22:34:21  2019-09-05 2019-09-05 22:34:00   
2  3.0  12.0  22.0 2019-09-04 22:34:21  2019-09-01 2019-09-01 22:34:00   

          c7  
0 2019-09-03  
1 2019-09-03  
2 2019-09-03  
    c1    c2    c

In [6]:
### 删除重复数据
### 默认重复数据仅保留第一次出现的数据
df.drop_duplicates() ### 一行数据都重复,才删除多余的数据
df.drop_duplicates(subset = "c2") ### c2一列重复了,就删除多余的行数据,相当于group by c2
### c1 和 c2两列重复了,就删除多余的行数据,相当于group by c1 , c2
## keep 选择保留哪些数据 first 保留第一次数据; last 保留最后一次数据 ;False全部删除,不保留任何数据,注意False 写成false是报错的
df.drop_duplicates(subset = ["c1","c2"],keep = False)


,c1,c2,c3,c4,c5,c6,c7
0,0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
1,1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
2,3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03
3,NaN,NaN,NaN,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [7]:
### 异常值检测和处理
### 业务规定指标的正常范围,通过阈值
### 是否正态分布,超过3倍标准差就是异常
### 绘制箱型图,大于 或者 小于 上下边缘的点称之为异常值
### 处理方式  删除、缺失值代替、专门研究异常值出现的原因
### 数据筛选--删除、筛选--替换 replace 在后面的章节介绍

In [8]:
### 数据类型 以及数据类型的转换 int float object string datetime unicode
df = df.dropna()
df["c1"].dtype  
df["c1"].astype("int64") ### float64 转换成 int64
df

,c1,c2,c3,c4,c5,c6,c7
0,0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
1,1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
2,3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [9]:
### 设置索引
df.set_index("c1")

,c2,c3,c4,c5,c6,c7
c1,,,,,,
0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [10]:
df.set_index(["c1","c2"]) ### 设置索引
### 重置索引 常用于 数据分组和透视表中
df.reset_index() ### 取消设置的索引
df.reset_index(level = 0) ### 将第0列设置为索引
df.reset_index(drop = True) ### 将现在的索引列删除

,c1,c2,c3,c4,c5,c6,c7
0,0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
1,1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
2,3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [11]:
df.rename(columns = {"c1":"cc1","c2":"cc2"},index = {0:"a",1:"b"}) ### 重置列名 和 索引名

,cc1,cc2,c3,c4,c5,c6,c7
a,0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
b,1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
2,3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [12]:
### 第6章 数据选择
print df["c1"] ### 选择一个列
print df[["c1","c2"]] ### 选择若干个列

0    0.0
1    1.0
2    3.0
Name: c1, dtype: float64
    c1    c2
0  0.0  10.0
1  1.0  11.0
2  3.0  12.0


In [13]:
### 选择某行、若干行
print df.iloc[0]  ## 获取第0行
print df.iloc[[0,2]] ## 获取第0、2行
print df.iloc[0:3] ## 获取第0到3行

c1                      0
c2                     10
c3                     20
c4    2019-09-04 22:34:21
c5             2019-09-03
c6    2019-09-03 22:34:00
c7    2019-09-03 00:00:00
Name: 0, dtype: object
    c1    c2    c3                  c4          c5                  c6  \
0  0.0  10.0  20.0 2019-09-04 22:34:21  2019-09-03 2019-09-03 22:34:00   
2  3.0  12.0  22.0 2019-09-04 22:34:21  2019-09-01 2019-09-01 22:34:00   

          c7  
0 2019-09-03  
2 2019-09-03  
    c1    c2    c3                  c4          c5                  c6  \
0  0.0  10.0  20.0 2019-09-04 22:34:21  2019-09-03 2019-09-03 22:34:00   
1  1.0  11.0  21.0 2019-09-04 22:34:21  2019-09-05 2019-09-05 22:34:00   
2  3.0  12.0  22.0 2019-09-04 22:34:21  2019-09-01 2019-09-01 22:34:00   

          c7  
0 2019-09-03  
1 2019-09-03  
2 2019-09-03  


In [14]:
df.iloc[0:2,0:2] ### 根据行列index选择,范围是[)

,c1,c2
0,0.0,10.0
1,1.0,11.0


In [15]:
print df
### 选择满足条件的行
df[ (df["c2"] >= 11) & (df["c3"] >= 22) ] ### 注意: ()必须填写,满足c2>=11 and c3 >=22的数据
df[ (df["c2"] >= 11) & (df["c3"] >= 22) ][["c5","c6"]] ### 选择某2列

    c1    c2    c3                  c4          c5                  c6  \
0  0.0  10.0  20.0 2019-09-04 22:34:21  2019-09-03 2019-09-03 22:34:00   
1  1.0  11.0  21.0 2019-09-04 22:34:21  2019-09-05 2019-09-05 22:34:00   
2  3.0  12.0  22.0 2019-09-04 22:34:21  2019-09-01 2019-09-01 22:34:00   

          c7  
0 2019-09-03  
1 2019-09-03  
2 2019-09-03  


,c5,c6
2,2019-09-01,2019-09-01 22:34:00


In [16]:
df.loc[0:2,["c1","c2"]]

,c1,c2
0,0.0,10.0
1,1.0,11.0
2,3.0,12.0


In [17]:
### 第7章 数值操作

In [18]:
df

,c1,c2,c3,c4,c5,c6,c7
0,0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
1,1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
2,3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [19]:
### 数值替换 replace
df.replace(10.0,"aa") ##所有列的数值,都要替换   数字可以替换成字符串、所有的10都替换成aa
df["c2"].replace(10.0,"aa") ###只是替换某一列的值
df["c2"].replace(np.NaN,0) ###相当于缺失值处理
df["c2"].replace([10,11],"aa") ### 将多个值 都替换成一个值
df["c2"].replace({10:"aa",11:"bb"}) ### 替换一组映射---map字典形式的参数
df

,c1,c2,c3,c4,c5,c6,c7
0,0.0,10.0,20.0,2019-09-04 22:34:21,2019-09-03,2019-09-03 22:34:00,2019-09-03
1,1.0,11.0,21.0,2019-09-04 22:34:21,2019-09-05,2019-09-05 22:34:00,2019-09-03
2,3.0,12.0,22.0,2019-09-04 22:34:21,2019-09-01,2019-09-01 22:34:00,2019-09-03


In [20]:
### 数据排序 sort_values 
### na_position 对NaN数据的位置进行排序,默认是last,其他值first
df = load_data('/Users/maming/Downloads/test/dataframe_2_data_deal.txt')
df.sort_values(by = "c3",ascending = False,na_position = "first") ### 倒序
df.sort_values(by = ["c6","c3"],ascending = [False,True],na_position = "first") ### 多列进行排序

,c1,c2,c3,c4,c5,c6,c7
1,1.0,11.0,21.0,20190904 22:34:21,2019-09-05,20190905 22:34:00,20190903
0,0.0,10.0,20.0,20190904 22:34:21,2019-09-03,20190903 22:34:00,20190903
3,NaN,NaN,NaN,20190904 22:34:21,2019-09-01,20190901 22:34:00,20190903
2,3.0,12.0,22.0,20190904 22:34:21,2019-09-01,20190901 22:34:00,20190903


In [21]:
### 数值删除
### 删除列
## df.drop("c2",1) ### 1表示删除列,0表示删除行
df.drop(["c2","c3"],axis = 1) ### 1表示删除列,0表示删除行
df.drop(columns = ["c4","c5"]) ### 或者columns参数写入要删除的列,这样就不用写 axis = 1

### 删除行
df.drop([0,1],axis = 0) ### 输入行号
df.drop(index = [1,2])

### 删除满足条件的行--保留条件为true的数据
df[df["c2"] > 10]

,c1,c2,c3,c4,c5,c6,c7
1,1.0,11.0,21.0,20190904 22:34:21,2019-09-05,20190905 22:34:00,20190903
2,3.0,12.0,22.0,20190904 22:34:21,2019-09-01,20190901 22:34:00,20190903


In [22]:
### 数值计数 value_counts 
print df["c6"].value_counts() ### 计算某一个列,每一个值出现的次数
print df["c6"].value_counts(normalize = True) ### 转换成占比
print df["c6"].value_counts(normalize = True,sort = False) ### 转换成频率,并且不需要按照占比排序

20190901 22:34:00    2
20190905 22:34:00    1
20190903 22:34:00    1
Name: c6, dtype: int64
20190901 22:34:00    0.50
20190905 22:34:00    0.25
20190903 22:34:00    0.25
Name: c6, dtype: float64
20190903 22:34:00    0.25
20190901 22:34:00    0.50
20190905 22:34:00    0.25
Name: c6, dtype: float64


In [23]:
### 某列的唯一值
df["c6"].unique()

array([u'20190903 22:34:00', u'20190905 22:34:00', u'20190901 22:34:00'],
      dtype=object)

In [24]:
### 数值查找,相当于contain,显示该数值是否在该表存在,或者该列存在
print df["c3"].isin([20,21])
print df.isin([20,21])
df[df["c3"].isin([20,21])] ### 仅保留c3列包含20和21值的数据
df[df.isin([20,21])] ## 这个无法过滤数据，只能约束某个列的基础上才能过滤数据


0     True
1     True
2    False
3    False
Name: c3, dtype: bool
      c1     c2     c3     c4     c5     c6     c7
0  False  False   True  False  False  False  False
1  False  False   True  False  False  False  False
2  False  False  False  False  False  False  False
3  False  False  False  False  False  False  False


,c1,c2,c3,c4,c5,c6,c7
0,NaN,NaN,20.0,NaN,NaN,NaN,NaN
1,NaN,NaN,21.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
### 无法插入新行,但是可以插入新列
df["new_column"] = df["c3"] * 2
df

,c1,c2,c3,c4,c5,c6,c7,new_column
0,0.0,10.0,20.0,20190904 22:34:21,2019-09-03,20190903 22:34:00,20190903,40.0
1,1.0,11.0,21.0,20190904 22:34:21,2019-09-05,20190905 22:34:00,20190903,42.0
2,3.0,12.0,22.0,20190904 22:34:21,2019-09-01,20190901 22:34:00,20190903,44.0
3,NaN,NaN,NaN,20190904 22:34:21,2019-09-01,20190901 22:34:00,20190903,NaN


In [26]:
### 数据转置
df.T

,0,1,2,3
c1,0,1,3,NaN
c2,10,11,12,NaN
c3,20,21,22,NaN
c4,20190904 22:34:21,20190904 22:34:21,20190904 22:34:21,20190904 22:34:21
c5,2019-09-03,2019-09-05,2019-09-01,2019-09-01
c6,20190903 22:34:00,20190905 22:34:00,20190901 22:34:00,20190901 22:34:00
c7,20190903,20190903,20190903,20190903
new_column,40,42,44,NaN


In [27]:
### apply 与 applymap函数
df = df.dropna()
print df

print df["c2"].apply(lambda x:x+1) ### apply针对某一列/某一行的数据,做数据转换
df = df.drop(["c4","c5","c6","c7"],axis = 1)
df
print df.applymap(lambda x:x+1) ### applymap针对df中所有的数据进行转换

    c1    c2    c3                 c4          c5                 c6  \
0  0.0  10.0  20.0  20190904 22:34:21  2019-09-03  20190903 22:34:00   
1  1.0  11.0  21.0  20190904 22:34:21  2019-09-05  20190905 22:34:00   
2  3.0  12.0  22.0  20190904 22:34:21  2019-09-01  20190901 22:34:00   

         c7  new_column  
0  20190903        40.0  
1  20190903        42.0  
2  20190903        44.0  
0    11.0
1    12.0
2    13.0
Name: c2, dtype: float64
    c1    c2    c3  new_column
0  1.0  11.0  21.0        41.0
1  2.0  12.0  22.0        43.0
2  4.0  13.0  23.0        45.0


In [28]:
### rank的method = min,max,first,average
df["c2"].rank(method = "average")

0    1.0
1    2.0
2    3.0
Name: c2, dtype: float64

In [29]:
### 区间切分bin---cut 与 pcut
### 先讲不与dataframe关联的知识
ages = [20,22,25,27,21,23,37,31,61,45,32]
bins = [18,25,35,60,100]
cats = pd.cut(ages,bins)
print(cats)
###输出内容是对ages每一个元素进行分组,分组到bins区间中
### Length: 11 说明ages一共11个元素
### bins5个元素,其实定义了四个区间(18, 25] < (25, 35] < (35, 60] < (60, 100]
### 第一行的结果表示每一个age最后被划分到哪个区间
### 返回的是Categories对象。
print cats.codes  ### [0 0 0 1 0 0 2 1 3 2 1] 可以将区间编码,打印编码会更清晰

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (35, 60], (25, 35], (60, 100], (35, 60], (25, 35]]
Length: 11
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]
[0 0 0 1 0 0 2 1 3 2 1]


In [30]:
### 可以对区间设置别名
group_names = ['Youth','YoungAdult','MiddleAged','Senior']
cats = pd.cut(ages,bins,labels = group_names) ### 可以对区间设置别名
print(cats)
print cats.codes

[Youth, Youth, Youth, YoungAdult, Youth, ..., MiddleAged, YoungAdult, Senior, MiddleAged, YoungAdult]
Length: 11
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]
[0 0 0 1 0 0 2 1 3 2 1]


In [31]:
### 可以统计每一个分布 对应的 元素数量,用于直方图等数据
type(pd.value_counts(cats.codes) ) ## 输出 Series
print pd.value_counts(cats.codes)
print pd.value_counts(cats.codes).index
print pd.value_counts(cats.codes).values

0    5
1    3
2    2
3    1
dtype: int64
Int64Index([0, 1, 2, 3], dtype='int64')
[5 3 2 1]


In [32]:
### 按照max min 对数据分成n等分,但不保证n等分的数据大小是相同的
cats = pd.cut(ages,4)
print(cats)
print cats.codes

[(19.959, 30.25], (19.959, 30.25], (19.959, 30.25], (19.959, 30.25], (19.959, 30.25], ..., (30.25, 40.5], (30.25, 40.5], (50.75, 61.0], (40.5, 50.75], (30.25, 40.5]]
Length: 11
Categories (4, interval[float64]): [(19.959, 30.25] < (30.25, 40.5] < (40.5, 50.75] < (50.75, 61.0]]
[0 0 0 0 0 0 1 1 3 2 1]


In [33]:
###qcut保证等长的拆分,因为他是用分位数进行分割的
data = np.random.randn(1000)  # 正态分布
cats = pd.qcut(data,4)       # 按四分位数进行切割
print(cats)
pd.value_counts(cats)

[(0.731, 3.631], (0.731, 3.631], (0.027, 0.731], (-0.674, 0.027], (-3.338, -0.674], ..., (0.731, 3.631], (0.731, 3.631], (-3.338, -0.674], (0.027, 0.731], (-3.338, -0.674]]
Length: 1000
Categories (4, interval[float64]): [(-3.338, -0.674] < (-0.674, 0.027] < (0.027, 0.731] < (0.731, 3.631]]


(0.731, 3.631]      250
(0.027, 0.731]      250
(-0.674, 0.027]     250
(-3.338, -0.674]    250
dtype: int64

In [34]:
###也通过指定分位数（0到1之间的数值，包含端点）进行面元划分
data = np.random.randn(1000)  # 正态分布
cats = pd.qcut(data,[0,0.1,0.5,0.9,1])
print(cats)
pd.value_counts(cats,sort = False) ## 可以看到数据是10% 40% 40% 10%

[(0.0103, 1.229], (-1.288, 0.0103], (-1.288, 0.0103], (-1.288, 0.0103], (0.0103, 1.229], ..., (-1.288, 0.0103], (-3.146, -1.288], (-1.288, 0.0103], (-1.288, 0.0103], (-1.288, 0.0103]]
Length: 1000
Categories (4, interval[float64]): [(-3.146, -1.288] < (-1.288, 0.0103] < (0.0103, 1.229] < (1.229, 3.131]]


(-3.146, -1.288]    100
(-1.288, 0.0103]    400
(0.0103, 1.229]     400
(1.229, 3.131]      100
dtype: int64

In [35]:
### cut与qcut的数据data是可以通过dataframe的某一个列获取到数据的
data = df["c2"]
cats = pd.qcut(data,[0,0.1,0.5,0.9,1])
print(cats)
pd.value_counts(cats,sort = False) ## 可以看到数据是10% 40% 40% 10%

0    (9.999, 10.2]
1     (10.2, 11.0]
2     (11.8, 12.0]
Name: c2, dtype: category
Categories (4, interval[float64]): [(9.999, 10.2] < (10.2, 11.0] < (11.0, 11.8] < (11.8, 12.0]]


(9.999, 10.2]    1
(10.2, 11.0]     1
(11.0, 11.8]     0
(11.8, 12.0]     1
Name: c2, dtype: int64